# Build the basin example base model

## Notebook Setup

In [ ]:
import os
import sys
import shutil
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pathlib as pl
from shapely.geometry import Polygon, LineString
import flopy
from flopy.discretization import StructuredGrid
import flopy.plot.styles as styles

In [ ]:
# import all plot style information from defaults.py
from defaults import *

In [ ]:
fine_topo = flopy.utils.Raster.load("./grid_data/fine_topo.asc")

In [ ]:
fine_topo.plot();

# Basin Example

In [ ]:
boundary_polygon = string2geom(boundary)
print("len boundary", len(boundary_polygon))
bp = np.array(boundary_polygon)

sgs = [
    string2geom(sg) for sg in (streamseg1, streamseg2, streamseg3, streamseg4)
]


fig = plt.figure(figsize=figsize)
ax = fig.add_subplot()
ax.set_aspect("equal")

riv_colors = ("blue", "cyan", "green", "orange", "red")

ax.plot(bp[:, 0], bp[:, 1], "ro-")
for idx, sg in enumerate(sgs):
    print("Len segment: ", len(sg))
    sa = np.array(sg)
    ax.plot(sa[:, 0], sa[:, 1], color=riv_colors[idx], lw=0.75, marker="o")

In [ ]:
def set_idomain(grid, boundary):
    from flopy.utils.gridintersect import GridIntersect
    from shapely.geometry import Polygon

    ix = GridIntersect(grid, method="vertex", rtree=True)
    result = ix.intersect(Polygon(boundary))
    idx = [coords for coords in result.cellids]
    idx = np.array(idx, dtype=int)
    nr = idx.shape[0]
    if idx.ndim == 1:
        idx = idx.reshape((nr, 1))
    print(idx.shape, idx.ndim)
    idx = tuple([idx[:, i] for i in range(idx.shape[1])])
    # idx = (idx[:, 0], idx[:, 1])
    idomain = np.zeros(grid.shape[1:], dtype=int)
    idomain[idx] = 1
    idomain = idomain.reshape(grid.shape)
    grid.idomain = idomain

In [ ]:
# Create a regular MODFLOW grid
dx = dy = 500.0  # 166.666666667
dv0 = 5.0
nlay = 1
nrow = int(Ly / dy) + 1
ncol = int(Lx / dx) + 1
print(Lx, Ly, nrow, ncol)
delr = np.array(ncol * [dx])
delc = np.array(nrow * [dy])
top = np.ones((nrow, ncol)) * 1000.0
botm = np.ones((nlay, nrow, ncol)) * -100.0
regular_grid = StructuredGrid(
    nlay=nlay, delr=delr, delc=delc, xoff=0.0, yoff=0.0, top=top, botm=botm
)

set_idomain(regular_grid, boundary_polygon)

In [ ]:
top_rg = fine_topo.resample_to_grid(
    regular_grid,
    band=fine_topo.bands[0],
    method="linear",
    extrapolate_edges=True,
)

In [ ]:
regular_grid.shape

In [ ]:
ixs = flopy.utils.GridIntersect(regular_grid, method="structured")
cellids = []
for sg in sgs:
    v = ixs.intersect(LineString(sg), sort_by_cellid=True)
    cellids += v["cellids"].tolist()
intersection_rg = np.zeros(regular_grid.shape[1:])
for loc in cellids:
    intersection_rg[loc] = 1

In [ ]:
with styles.USGSMap():
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot()
    pmv = flopy.plot.PlotMapView(modelgrid=regular_grid)
    ax.set_aspect("equal")
    pmv.plot_array(top_rg)
    pmv.plot_array(
        intersection_rg,
        masked_values=[
            0,
        ],
        alpha=0.2,
        cmap="Reds_r",
    )
    # pmv.plot_grid(lw=0.25, color="0.5")
    pmv.plot_inactive()
    # regular_grid.plot(ax=ax, )
    ax.plot(bp[:, 0], bp[:, 1], "r-")
    for sg in sgs:
        sa = np.array(sg)
        ax.plot(sa[:, 0], sa[:, 1], "b-")

### Build the base model

In [ ]:
sim_ws = pl.Path("../examples/ex-basin/basin_base")

In [ ]:
# redefine the number of layers
nlay = 5

In [ ]:
newton_options = "NEWTON UNDER_RELAXATION"
linear_accelerator = "bicgstab"
outer_maximum = 1000
rewet_record = None
wetdry = None

In [ ]:
drn_intersection = []
drn_cellids = []
drn_lengths = []
for sg in sgs:
    v = ixs.intersect(LineString(sg), sort_by_cellid=True)
    drn_intersection.append(v)
    drn_cellids += v["cellids"].tolist()
    drn_lengths += v["lengths"].tolist()

In [ ]:
leakance = 1.0 / (0.5 * dv0)  # kv / b
drn_data = []
for (r, c), length in zip(drn_cellids, drn_lengths):
    x = regular_grid.xcellcenters[r, c]
    width = 5.0 + (14.0 / Lx) * (Lx - x)
    conductance = leakance * length * width
    drn_data.append((0, r, c, top_rg[r, c], conductance))
drn_data[:10]

In [ ]:
# groundwater discharge to surface
idomain = regular_grid.idomain[0].copy()
index = tuple(np.array(list(zip(*drn_cellids))))
idomain[index] = -1
gw_discharge_data = []
for r in range(nrow):
    for c in range(ncol):
        if idomain[r, c] < 1:
            continue
        conductance = leakance * dx * dy
        gw_discharge_data.append(
            (0, r, c, top_rg[r, c] - 0.5, conductance, 1.0)
        )
gw_discharge_data[:10]

In [ ]:
plt.imshow(idomain);

In [ ]:
topc = np.zeros((nlay, nrow, ncol), dtype=float)
botm = np.zeros((nlay, nrow, ncol), dtype=float)
dv = dv0
topc[0] = top_rg.copy()
botm[0] = topc[0] - dv
for idx in range(1, nlay):
    dv *= 1.5
    topc[idx] = botm[idx - 1]
    botm[idx] = topc[idx] - dv

In [ ]:
for k in range(nlay):
    print((topc[k] - botm[k]).mean())

In [ ]:
idomain = np.array(
    [regular_grid.idomain[0, :, :].copy() for k in range(nlay)], dtype=int
)
strt = np.array([top_rg.copy() for k in range(nlay)], dtype=float)

### Build the model files using FloPy

In [ ]:
sim = flopy.mf6.MFSimulation(
    sim_name="basin",
    sim_ws=sim_ws,
    exe_name="mf6",
)

tdis = flopy.mf6.ModflowTdis(sim)
ims = flopy.mf6.ModflowIms(
    sim,
    complexity="simple",
    print_option="SUMMARY",
    linear_acceleration=linear_accelerator,
    outer_maximum=outer_maximum,
    inner_maximum=100,
    outer_dvclose=1e-5,
    inner_dvclose=1e-6,
    preconditioner_levels=2,
    relaxation_factor=0.0,
)
gwf = flopy.mf6.ModflowGwf(sim, save_flows=True, newtonoptions=newton_options)

dis = flopy.mf6.ModflowGwfdis(
    gwf,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=dx,
    delc=dy,
    idomain=idomain,
    top=top_rg,
    botm=botm,
    xorigin=0.0,
    yorigin=0.0,
)

ic = flopy.mf6.ModflowGwfic(gwf, strt=strt)
npf = flopy.mf6.ModflowGwfnpf(
    gwf,
    save_specific_discharge=True,
    icelltype=1,
    k=1.0,
    rewet_record=rewet_record,
    wetdry=wetdry,
)
sto = flopy.mf6.ModflowGwfsto(
    gwf,
    iconvert=1,
    ss=1e-5,
    sy=0.2,
    steady_state=True,
)
rch = flopy.mf6.ModflowGwfrcha(
    gwf,
    recharge=0.000001,
)
drn = flopy.mf6.ModflowGwfdrn(
    gwf,
    stress_period_data=drn_data,
    pname="river",
)
drn_gwd = flopy.mf6.ModflowGwfdrn(
    gwf,
    auxiliary=["depth"],
    auxdepthname="depth",
    stress_period_data=gw_discharge_data,
    pname="gwd",
)
oc = flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord=f"{gwf.name}.hds",
    budget_filerecord=f"{gwf.name}.cbc",
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
    printrecord=[("BUDGET", "ALL")],
)

### Write the model files

In [ ]:
sim.write_simulation()

In [ ]:
write_petscdb(sim_ws)

### Run the model

In [ ]:
if run_simulation():
    sim.run_simulation()